In [1]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
import re
import xml.etree.cElementTree as ET
import csv
import cerberus
import schema
import codecs



osm_file = open("Houston_Texas_sample.osm", "r")

#

#########################################

# Search for all unexpected street types

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)



# List of expected street name values
expected = ["Street", "Avenue", "Lane", "Way", "Boulevard",  
"Drive", "Court", "Place", "Square", "Road", "Trail", "Parkway", "Commons", "North", "South", "West", "East"]

# Here we add any street name that isn't in our expected dictionary
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
    return street_types

# Here we create a dictionary of our postal codes
def audit_postal_code(postal_code_types, postal_code):  
    if not postal_code.isupper() or ' ' not in postal_code:
        postal_code_types['case_whitespace_problems'].add(postal_code)
    else:
        postal_code_types['other'].add(postal_code)
    return postal_code_types



def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def is_postal_code(elem):
    return (elem.attrib['k'] == "addr:postcode")

#audit(osm_file)
#print_sorted_dict(street_types)

def audit(filename):
    f = (filename)
    street_types = defaultdict(set)
    postal_code_types = defaultdict(set)
    
    for event, element in ET.iterparse(f, events=("start",)):
        if element.tag == "node" or element.tag =="way":
            for tag in element.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
                if is_postal_code(tag):
                    audit_postal_code(postal_code_types, tag.attrib['v'])
                    
    f.close()
    return dict(street_types), dict(postal_code_types)

mapping = { "St": "Street",
            "St.": "Street",
            "Ave" : "Avenue",
            "Ave.": "Avenue",
            "Rd.": "Road",
            "Rd": "Road",
            "E" : "East",
            "W" : "West",
            "S" : "South",
            "N" : "North"
            }

# Here we iterate over each name and compare/update it in regards to our selected mapping
def update_name(street_name, mapping):
    street_name = street_name.replace(' ', ' ')
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        street_type2 = ' '.join(street_name.split()[0:])
        if street_type in mapping.keys():
            #print 'Before: ' , name
            street_name = re.sub(street_type, mapping[street_type],street_name)
            #print 'After: ', name
        
    return street_name

# Here we iterate over the postal codes making them a uniform 5 digit number
def update_postal_code(postal_code):
    postal_code = postal_code.upper()
    if ' ' not in postal_code:
        if len(postal_code) != 5:
            postal_code = postal_code[0:5]
    return postal_code


def street_test():
    
    st_types = audit(osm_file)[0]
    print ("")
    for st_types, st_names in st_types.iteritems():
        for name in st_names:
            better_name = update_name(name, mapping)
            print (name, "->", better_name)
            
def postal_code_test():
    postcode_types = audit(osm_file)[1]
    #pprint(postcode_types)
    print ("")
    
    for postcode_type, postcodes in postcode_types.iteritems():
        for postcode in postcodes:
            better_postcode = update_postal_code(postcode)
            print (postcode, "=>", better_postcode)